# 04a - Model Development Master
# 
# **Coordinador Principal de la Fase 4: Desarrollo de Modelos**
# 
# Este notebook orquesta todo el desarrollo de modelos para el proyecto de Monitorización Multimodal de Alzheimer:
# 
# **Objetivos principales**:
# - Coordinar la ejecución de todos los pipelines de modelado
# - Integrar resultados de regresión, clasificación, análisis temporal y estratificación
# - Gestionar experimentos MLflow de manera centralizada
# - Generar resumen executivo de todos los modelos desarrollados
# 
# **Notebooks integrados**:
# - 04b_regression_models.ipynb (composite_risk_score)
# - 04c_classification_models.ipynb (risk_category)
# - 04d_temporal_analysis.ipynb (series temporales)
# - 04e_risk_stratification.ipynb (estratificación)

## Importar librerías

In [1]:
import sys
import os
sys.path.append('../src/modeling')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Importar scripts de modelado
from regression_pipeline import RegressionPipeline
from classification_pipeline import ClassificationPipeline
from temporal_modeling import TemporalModeling
from risk_stratification import RiskStratification
from ensemble_methods import EnsembleMethods
from model_utils import ModelUtils

In [2]:
# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

Iniciando Fase 4: Desarrollo de Modelos


In [ ]:
print("🚀 FASE 4: DESARROLLO DE MODELOS - COORDINADOR MASTER")
print("=" * 60)
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ Librerías y scripts importados correctamente")

## Configuración MLflow Master

In [3]:
# Configuración MLflow Master
mlflow.set_experiment("alzheimer_model_development_master")

# Configuración global del proyecto
PROJECT_CONFIG = {
    'project_name': 'Alzheimer_Multimodal_Monitoring',
    'phase': 'model_development',
    'target_regression': 'composite_risk_score',
    'target_classification': 'risk_category',
    'data_path': '../data/processed/integrated_features_final.csv',
    'results_path': '../results/model_development/',
    'random_state': 42
}

# Crear directorio de resultados si no existe
os.makedirs(PROJECT_CONFIG['results_path'], exist_ok=True)

print("🔧 Configuración MLflow Master establecida")
print(f"🎯 Target Regresión: {PROJECT_CONFIG['target_regression']}")
print(f"🎯 Target Clasificación: {PROJECT_CONFIG['target_classification']}")


📁 Configurando rutas y cargando datos...
📊 Cargando dataset final...
📋 Cargando metadatos de feature engineering...
✅ Dataset cargado: (48466, 189)
✅ Features seleccionadas: 192
✅ Registros válidos: 48466


## Cargar y validar datos

In [5]:
# Cargar y validar datos
print("\n📊 CARGA Y VALIDACIÓN DE DATOS")
print("-" * 40)

try:
    df = pd.read_csv(PROJECT_CONFIG['data_path'])
    
    # Validaciones básicas
    utils = ModelUtils()
    data_quality = utils.validate_data_quality(df, 
                                               PROJECT_CONFIG['target_regression'],
                                               PROJECT_CONFIG['target_classification'])
    
    print(f"✅ Dataset cargado: {df.shape}")
    print(f"📈 Registros válidos para regresión: {data_quality['regression_samples']}")
    print(f"📊 Registros válidos para clasificación: {data_quality['classification_samples']}")
    print(f"🎯 Features disponibles: {data_quality['total_features']}")
    print(f"📉 Completitud promedio: {data_quality['data_completeness']:.1%}")
    
    if data_quality['quality_score'] < 0.7:
        print("⚠️  Advertencia: Calidad de datos por debajo del umbral recomendado")
    
except FileNotFoundError:
    print("❌ Error: Archivo de features no encontrado")
    print("💡 Ejecuta primero el notebook 03_feature_engineering_master.ipynb")
    raise


🔧 Configurando MLflow para tracking de experimentos...
✅ Experimento creado: Alzheimer_Multimodal_Monitoring_Phase4


## Inicializar pipelines de modelado

In [6]:
# Inicializar pipelines de modelado
print("\n🔧 INICIALIZACIÓN DE PIPELINES")
print("-" * 40)

with mlflow.start_run(run_name="master_pipeline_initialization"):
    mlflow.set_tag("phase", "model_development")
    mlflow.set_tag("pipeline_type", "master_coordinator")
    
    # Inicializar pipelines
    regression_pipeline = RegressionPipeline(random_state=PROJECT_CONFIG['random_state'])
    classification_pipeline = ClassificationPipeline(random_state=PROJECT_CONFIG['random_state'])
    temporal_pipeline = TemporalModeling(random_state=PROJECT_CONFIG['random_state'])
    stratification_pipeline = RiskStratification(random_state=PROJECT_CONFIG['random_state'])
    ensemble_pipeline = EnsembleMethods(random_state=PROJECT_CONFIG['random_state'])
    
    # Registrar configuración
    mlflow.log_params(PROJECT_CONFIG)
    mlflow.log_metrics(data_quality)
    
    print("✅ Pipeline de Regresión inicializado")
    print("✅ Pipeline de Clasificación inicializado") 
    print("✅ Pipeline Temporal inicializado")
    print("✅ Pipeline de Estratificación inicializado")
    print("✅ Pipeline de Ensemble inicializado")



📊 Realizando análisis preliminar del dataset...
🔍 INFORMACIÓN BÁSICA DEL DATASET:
  • Forma del dataset: (48466, 189)
  • Registros únicos: 48198
  • Memoria utilizada: 92.6 MB

🎯 VARIABLES OBJETIVO:
  • Continua: composite_risk_score
  • Categórica: risk_category

📈 DISTRIBUCIÓN - COMPOSITE_RISK_SCORE:
  • count: 48466.0000
  • mean: 0.3671
  • std: 0.2128
  • min: 0.0000
  • 25%: 0.1489
  • 50%: 0.3631
  • 75%: 0.5714
  • max: 0.9286

📊 DISTRIBUCIÓN - RISK_CATEGORY:
  • Low: 22,501 (46.4%)
  • Moderate: 22,345 (46.1%)
  • High: 3,620 (7.5%)


## 1. EJECUTAR MODELOS DE REGRESIÓN

In [ ]:
# 1. EJECUTAR MODELOS DE REGRESIÓN
print("\n🎯 FASE 1: MODELOS DE REGRESIÓN")
print("-" * 40)

with mlflow.start_run(run_name="regression_models_execution", nested=True):
    mlflow.set_tag("model_family", "regression")
    
    # Preparar datos para regresión
    X_reg, y_reg = utils.prepare_regression_data(df, PROJECT_CONFIG['target_regression'])
    
    # Ejecutar pipeline de regresión
    regression_results = regression_pipeline.run_full_pipeline(X_reg, y_reg)
    
    # Registrar mejores métricas
    best_model = regression_results['best_model']
    mlflow.log_metrics({
        'best_regression_r2': regression_results['best_metrics']['r2'],
        'best_regression_rmse': regression_results['best_metrics']['rmse'],
        'best_regression_mae': regression_results['best_metrics']['mae']
    })
    
    print(f"🏆 Mejor modelo de regresión: {best_model}")
    print(f"📊 R² Score: {regression_results['best_metrics']['r2']:.4f}")
    print(f"📊 RMSE: {regression_results['best_metrics']['rmse']:.4f}")


## 2. EJECUTAR MODELOS DE CLASIFICACIÓN

In [ ]:
# 2. EJECUTAR MODELOS DE CLASIFICACIÓN  
print("\n🎯 FASE 2: MODELOS DE CLASIFICACIÓN")
print("-" * 40)

with mlflow.start_run(run_name="classification_models_execution", nested=True):
    mlflow.set_tag("model_family", "classification")
    
    # Preparar datos para clasificación
    X_clf, y_clf = utils.prepare_classification_data(df, PROJECT_CONFIG['target_classification'])
    
    # Ejecutar pipeline de clasificación
    classification_results = classification_pipeline.run_full_pipeline(X_clf, y_clf)
    
    # Registrar mejores métricas
    best_clf_model = classification_results['best_model']
    mlflow.log_metrics({
        'best_classification_accuracy': classification_results['best_metrics']['accuracy'],
        'best_classification_f1': classification_results['best_metrics']['f1_macro'],
        'best_classification_precision': classification_results['best_metrics']['precision_macro'],
        'best_classification_recall': classification_results['best_metrics']['recall_macro']
    })
    
    print(f"🏆 Mejor modelo de clasificación: {best_clf_model}")
    print(f"📊 Accuracy: {classification_results['best_metrics']['accuracy']:.4f}")
    print(f"📊 F1-Score: {classification_results['best_metrics']['f1_macro']:.4f}")


## 3. EJECUTAR ANÁLISIS TEMPORAL

In [ ]:
# 3. EJECUTAR ANÁLISIS TEMPORAL
print("\n🎯 FASE 3: ANÁLISIS TEMPORAL")
print("-" * 40)

with mlflow.start_run(run_name="temporal_analysis_execution", nested=True):
    mlflow.set_tag("model_family", "temporal")
    
    # Verificar disponibilidad de datos temporales
    temporal_features = [col for col in df.columns if any(x in col.lower() for x in ['time', 'date', 'sequence', 'visit'])]
    
    if temporal_features:
        # Ejecutar análisis temporal
        temporal_results = temporal_pipeline.run_temporal_analysis(df, temporal_features)
        
        mlflow.log_metrics({
            'temporal_features_count': len(temporal_features),
            'temporal_samples': temporal_results.get('samples', 0)
        })
        
        print(f"📈 Features temporales identificadas: {len(temporal_features)}")
        print(f"📊 Análisis temporal completado")
    else:
        print("⚠️  No se encontraron features temporales suficientes")
        print("💡 Análisis temporal omitido en esta iteración")


## 4. EJECUTAR ESTRATIFICACIÓN DE RIESGO

In [ ]:
# 4. EJECUTAR ESTRATIFICACIÓN DE RIESGO
print("\n🎯 FASE 4: ESTRATIFICACIÓN DE RIESGO")
print("-" * 40)

with mlflow.start_run(run_name="risk_stratification_execution", nested=True):
    mlflow.set_tag("model_family", "stratification")
    
    # Ejecutar estratificación
    stratification_results = stratification_pipeline.run_stratification_analysis(df, 
                                                                               PROJECT_CONFIG['target_regression'])
    
    # Registrar métricas de estratificación
    mlflow.log_metrics({
        'phenotypes_identified': stratification_results['n_phenotypes'],
        'stratification_quality': stratification_results['silhouette_score'],
        'risk_groups': stratification_results['n_risk_groups']
    })
    
    print(f"🧬 Fenotipos identificados: {stratification_results['n_phenotypes']}")
    print(f"📊 Calidad de estratificación: {stratification_results['silhouette_score']:.3f}")
    print(f"🎯 Grupos de riesgo: {stratification_results['n_risk_groups']}")


## 5. EJECUTAR MÉTODOS ENSEMBLE

In [ ]:
# 5. EJECUTAR MÉTODOS ENSEMBLE
print("\n🎯 FASE 5: MÉTODOS ENSEMBLE")
print("-" * 40)

with mlflow.start_run(run_name="ensemble_methods_execution", nested=True):
    mlflow.set_tag("model_family", "ensemble")
    
    # Combinar mejores modelos en ensemble
    base_models = {
        'regression': regression_results['top_models'][:3],
        'classification': classification_results['top_models'][:3]
    }
    
    # Ejecutar ensemble para regresión
    ensemble_reg_results = ensemble_pipeline.create_regression_ensemble(X_reg, y_reg, base_models['regression'])
    
    # Ejecutar ensemble para clasificación  
    ensemble_clf_results = ensemble_pipeline.create_classification_ensemble(X_clf, y_clf, base_models['classification'])
    
    # Registrar métricas ensemble
    mlflow.log_metrics({
        'ensemble_regression_r2': ensemble_reg_results['ensemble_score'],
        'ensemble_classification_f1': ensemble_clf_results['ensemble_score'],
        'ensemble_improvement_reg': ensemble_reg_results['improvement'],
        'ensemble_improvement_clf': ensemble_clf_results['improvement']
    })
    
    print(f"🎯 Ensemble Regresión R²: {ensemble_reg_results['ensemble_score']:.4f}")
    print(f"🎯 Ensemble Clasificación F1: {ensemble_clf_results['ensemble_score']:.4f}")
    print(f"📈 Mejora Regresión: +{ensemble_reg_results['improvement']:.3f}")
    print(f"📈 Mejora Clasificación: +{ensemble_clf_results['improvement']:.3f}")


## RESUMEN EJECUTIVO DE RESULTADOS

In [ ]:
# RESUMEN EJECUTIVO DE RESULTADOS
print("\n" + "="*60)
print("📊 RESUMEN EJECUTIVO - DESARROLLO DE MODELOS")
print("="*60)

# Compilar resultados finales
final_results = {
    'Datos': {
        'Total_Registros': len(df),
        'Features_Finales': len([col for col in df.columns if col not in [PROJECT_CONFIG['target_regression'], PROJECT_CONFIG['target_classification']]]),
        'Completitud_Datos': f"{data_quality['data_completeness']:.1%}",
        'Calidad_Score': f"{data_quality['quality_score']:.3f}"
    },
    'Modelos_Regresión': {
        'Mejor_Modelo': best_model,
        'R2_Score': f"{regression_results['best_metrics']['r2']:.4f}",
        'RMSE': f"{regression_results['best_metrics']['rmse']:.4f}",
        'Modelos_Evaluados': len(regression_results['all_results'])
    },
    'Modelos_Clasificación': {
        'Mejor_Modelo': best_clf_model,
        'Accuracy': f"{classification_results['best_metrics']['accuracy']:.4f}",
        'F1_Score': f"{classification_results['best_metrics']['f1_macro']:.4f}",
        'Modelos_Evaluados': len(classification_results['all_results'])
    },
    'Estratificación': {
        'Fenotipos_Identificados': stratification_results['n_phenotypes'],
        'Grupos_Riesgo': stratification_results['n_risk_groups'],
        'Calidad_Clustering': f"{stratification_results['silhouette_score']:.3f}"
    },
    'Ensemble': {
        'Mejora_Regresión': f"+{ensemble_reg_results['improvement']:.3f}",
        'Mejora_Clasificación': f"+{ensemble_clf_results['improvement']:.3f}",
        'Score_Final_R2': f"{ensemble_reg_results['ensemble_score']:.4f}",
        'Score_Final_F1': f"{ensemble_clf_results['ensemble_score']:.4f}"
    }
}

# Mostrar resumen
for category, metrics in final_results.items():
    print(f"\n🎯 {category.upper().replace('_', ' ')}:")
    for metric, value in metrics.items():
        print(f"   • {metric.replace('_', ' ')}: {value}")


## Guardar resultados y Generar reportes

In [ ]:
# Guardar resultados y generar reportes
print("\n📁 GUARDADO DE RESULTADOS Y REPORTES")
print("-" * 40)

# Compilar todos los resultados
master_results = {
    'execution_timestamp': datetime.now().isoformat(),
    'project_config': PROJECT_CONFIG,
    'data_quality': data_quality,
    'regression_results': regression_results,
    'classification_results': classification_results,
    'stratification_results': stratification_results,
    'ensemble_results': {
        'regression': ensemble_reg_results,
        'classification': ensemble_clf_results
    },
    'final_summary': final_results
}

# Guardar resultados master
import json
results_file = os.path.join(PROJECT_CONFIG['results_path'], 'master_results.json')
with open(results_file, 'w') as f:
    # Convertir numpy arrays a listas para serialización JSON
    serializable_results = utils.make_json_serializable(master_results)
    json.dump(serializable_results, f, indent=2)

print(f"✅ Resultados guardados en: {results_file}")

# Generar reporte CSV para análisis posterior
results_df = pd.DataFrame({
    'Pipeline': ['Regression', 'Classification', 'Stratification', 'Ensemble_Reg', 'Ensemble_Clf'],
    'Best_Score': [
        regression_results['best_metrics']['r2'],
        classification_results['best_metrics']['f1_macro'],
        stratification_results['silhouette_score'],
        ensemble_reg_results['ensemble_score'],
        ensemble_clf_results['ensemble_score']
    ],
    'Models_Evaluated': [
        len(regression_results['all_results']),
        len(classification_results['all_results']), 
        stratification_results['n_phenotypes'],
        len(base_models['regression']),
        len(base_models['classification'])
    ]
})

csv_file = os.path.join(PROJECT_CONFIG['results_path'], 'pipeline_summary.csv')
results_df.to_csv(csv_file, index=False)
print(f"✅ Resumen CSV guardado en: {csv_file}")


## Estado final y Próximos pasos

In [ ]:
# Estado final y próximos pasos
print("\n" + "="*60)
print("🎉 FASE 4 COMPLETADA EXITOSAMENTE")
print("="*60)

completion_status = {
    '✅ Modelos de Regresión': f"{len(regression_results['all_results'])} modelos evaluados",
    '✅ Modelos de Clasificación': f"{len(classification_results['all_results'])} modelos evaluados",
    '✅ Análisis Temporal': "Completado" if temporal_features else "Omitido (sin datos temporales)",
    '✅ Estratificación de Riesgo': f"{stratification_results['n_phenotypes']} fenotipos identificados",
    '✅ Métodos Ensemble': "Mejoras significativas obtenidas",
    '✅ Resultados Guardados': "JSON y CSV generados",
    '✅ Experimentos MLflow': "Todos registrados correctamente"
}

print("\n📋 ESTADO DE COMPLETITUD:")
for status, detail in completion_status.items():
    print(f"   {status}: {detail}")

print(f"\n🔄 PRÓXIMA FASE: Evaluación y Optimización")
print(f"📂 Resultados disponibles en: {PROJECT_CONFIG['results_path']}")
print(f"🔬 Experimentos MLflow: alzheimer_model_development_master")

print(f"\n💡 RECOMENDACIONES PARA FASE 5:")
print(f"   • Revisar métricas de validación cruzada")
print(f"   • Optimizar hiperparámetros de mejores modelos")
print(f"   • Evaluar explicabilidad de modelos")
print(f"   • Validar en conjunto de prueba independiente")

print("\n🚀 ¡Listo para continuar con la Fase 5!")

---

__Abraham Tartalos__